In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [2]:
# windows
df = pd.read_excel('E:\PHD\sucide_project\BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [17]:
'''
data processing issues：
1. column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
2. after 1 Psychiatric, data has some missing value
'''
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']


In [18]:
# divide into 2 categories
"""
dibide into 2 categories
Category 1: High Risk

Those with suicidal ideation (having thoughts of suicide) and those with a suicide plan (having a plan to carry out suicide) can be grouped into a high-risk category as they are actively considering and planning suicide.
Category 2: Past Attempts/ Hospitalizations

Those with a history of suicide attempt(s) and those with hospitalization(s) for suicidality can be grouped into a second category as they have a history of suicidal behavior but are not currently at an elevated risk of suicide.

"""
new_y = []
for i in y:
    if i == 0 or i ==1:
        new_y.append(0)
    else:
        new_y.append(1)


In [19]:
# train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=10)
print(len(X_train), len(X_test))


505 249


In [35]:
#　使用未进行参数调整的logistic regression , svm, random forest, xgboost
from sklearn.linear_model import LogisticRegression as LR
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

X = StandardScaler().fit_transform(x)

l1_ratio = 0.5
C_list = [1, 0.1, 0.01]
for C in C_list:
    clf_l1_LR = LR(C=C, penalty="l1", tol=0.01, solver="saga")
    clf_l2_LR = LR(C=C, penalty="l2", tol=0.01, solver="saga")
    clf_en_LR = LR(
        C=C, penalty="elasticnet", solver="saga", l1_ratio=l1_ratio, tol=0.01
    )
    clf_l1_LR.fit(X_train, y_train)
    clf_l2_LR.fit(X_train, y_train)
    clf_en_LR.fit(X_train, y_train)
    y_pred_l1 = clf_l1_LR.predict(X_test)
    y_pred_l2 = clf_l2_LR.predict(X_test)
    y_pred_en = clf_en_LR.predict(X_test)

    print("Score with L1 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred_l1), precision_score(y_test, y_pred_l1), recall_score(y_test, y_pred_l1), f1_score(y_test, y_pred_l1), roc_auc_score(y_test, y_pred_l1)))
    print("Score with Elastic-Net penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred_en), precision_score(y_test, y_pred_en), recall_score(y_test, y_pred_en), f1_score(y_test, y_pred_en), roc_auc_score(y_test, y_pred_en)))
    print("Score with L2 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred_l2), precision_score(y_test, y_pred_l2), recall_score(y_test, y_pred_l2), f1_score(y_test, y_pred_l2), roc_auc_score(y_test, y_pred_l2)))


Score with L1 penalty: accuracy:0.8112449799196787 precision:0.6666666666666666 recall:0.08163265306122448 f1:0.14545454545454545 roc_auc:0.5358163265306123
Score with Elastic-Net penalty: accuracy:0.8152610441767069 precision:0.7142857142857143 recall:0.10204081632653061 f1:0.17857142857142858 roc_auc:0.5460204081632652
Score with L2 penalty: accuracy:0.8192771084337349 precision:0.7 recall:0.14285714285714285 f1:0.23728813559322035 roc_auc:0.5639285714285714
Score with L1 penalty: accuracy:0.8032128514056225 precision:0.0 recall:0.0 f1:0.0 roc_auc:0.5
Score with Elastic-Net penalty: accuracy:0.8032128514056225 precision:0.0 recall:0.0 f1:0.0 roc_auc:0.5
Score with L2 penalty: accuracy:0.8032128514056225 precision:0.5 recall:0.02040816326530612 f1:0.039215686274509796 roc_auc:0.5077040816326531
Score with L1 penalty: accuracy:0.8032128514056225 precision:0.0 recall:0.0 f1:0.0 roc_auc:0.5
Score with Elastic-Net penalty: accuracy:0.8032128514056225 precision:0.0 recall:0.0 f1:0.0 roc_au

/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages

In [32]:
# 调整参数
from sklearn.model_selection import GridSearchCV
model = LR()
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[0.001, 0.01, 0.1, 1, 10, 100],
              'solver':['liblinear', 'sag', 'sega']}
grid_search = GridSearchCV(model, param_grid, cv=5)



Score with L1 penalty: accuracy:0.8112449799196787 precision:0.6666666666666666 recall:0.08163265306122448 f1:0.14545454545454545 roc_auc:0.5358163265306123
Score with L1 penalty: accuracy:0.8112449799196787 precision:0.6666666666666666 recall:0.08163265306122448 f1:0.14545454545454545 roc_auc:0.5358163265306123


In [36]:
print(len(X))
print(len(y))


754
754


In [6]:
# evaluation metric
from sklearn.metrics import accuracy_score
y_pred = clf_l1_LR.predict(X)
print('Accuracy:{}'.format(accuracy_score(y, y_pred)))

Accuracy:0.7254641909814323
